In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from numpy import pi
import re

# 공통 구조: 기능적 세포 1단위
def functional_cell():
    cell = QuantumCircuit(2)
    cell.h(0)
    cell.cx(0, 1)
    cell.ry(pi / 4, 1)
    cell.cx(0, 1)
    return cell

# 회로 케이스 A: 기본 구조 반복 (baseline)
def circuit_case_A():
    qc = QuantumCircuit(2)
    for _ in range(3):
        qc.compose(functional_cell(), inplace=True)
    qc.measure_all()
    return qc

# 회로 케이스 B: 구조 사이 barrier 삽입
def circuit_case_B():
    qc = QuantumCircuit(2)
    for _ in range(3):
        qc.compose(functional_cell(), inplace=True)
        qc.barrier()
    qc.measure_all()
    return qc

# 회로 케이스 C: 구조 사이 Z(π/2) 삽입 (간섭 유도)
def circuit_case_C():
    qc = QuantumCircuit(2)
    for i in range(3):
        qc.compose(functional_cell(), inplace=True)
        if i < 2:  # 마지막 반복 제외
            qc.rz(pi / 2, 1)
    qc.measure_all()
    return qc

# 회로 모음
circuits = {
    "Case A: Baseline": circuit_case_A(),
    "Case B: Barrier": circuit_case_B(),
    "Case C: Z(π/2) Inserted": circuit_case_C()
}

# 시뮬레이션 실행
simulator = AerSimulator()
results = {}

for label, circuit in circuits.items():
    job = simulator.run(circuit, shots=1024)
    result = job.result()
    counts = result.get_counts()
    results[label] = counts

# 결과 시각화
for label, counts in results.items():
    plot_histogram(counts, title=label)
    plt.show()

# 결과 시각화 및 안전한 파일 저장
for label, counts in results.items():
    fig = plot_histogram(counts, title=label)
    
    # 파일명 안전하게 정제 (공백, 괄호, 특수문자 제거)
    filename = re.sub(r"[^\w\-]+", "_", label.lower()) + ".png"
    
    fig.savefig(filename)
    plt.close(fig)


In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from numpy import pi
import numpy as np
from scipy.spatial.distance import cosine
import re

# 공통 구조: 기능적 세포 1단위
def functional_cell():
    cell = QuantumCircuit(2)
    cell.h(0)
    cell.cx(0, 1)
    cell.ry(pi / 4, 1)
    cell.cx(0, 1)
    return cell

# 회로 케이스 A: 기본 구조 반복 (baseline)
def circuit_case_A():
    qc = QuantumCircuit(2)
    for _ in range(3):
        qc.compose(functional_cell(), inplace=True)
    qc.measure_all()
    return qc

# 회로 케이스 B: 구조 사이 barrier 삽입
def circuit_case_B():
    qc = QuantumCircuit(2)
    for _ in range(3):
        qc.compose(functional_cell(), inplace=True)
        qc.barrier()
    qc.measure_all()
    return qc

# 회로 케이스 C: 구조 사이 Z(π/2) 삽입 (간섭 유도)
def circuit_case_C():
    qc = QuantumCircuit(2)
    for i in range(3):
        qc.compose(functional_cell(), inplace=True)
        if i < 2:
            qc.rz(pi / 2, 1)
    qc.measure_all()
    return qc

# 정량 지표 함수 정의
def shannon_entropy(counts):
    total = sum(counts.values())
    probs = [v / total for v in counts.values()]
    return -sum(p * np.log2(p) for p in probs if p > 0)

def distribution_vector(counts, keys):
    total = sum(counts.values())
    return np.array([counts.get(k, 0) / total for k in keys])

# 회로 모음
circuits = {
    "Case A: Baseline": circuit_case_A(),
    "Case B: Barrier": circuit_case_B(),
    "Case C: Z(π/2) Inserted": circuit_case_C()
}

# 시뮬레이션 실행
simulator = AerSimulator()
results = {}

for label, circuit in circuits.items():
    job = simulator.run(circuit, shots=1024)
    result = job.result()
    counts = result.get_counts()
    results[label] = counts

# 결과 시각화
for label, counts in results.items():
    plot_histogram(counts, title=label)
    plt.show()

# 결과 저장
for label, counts in results.items():
    fig = plot_histogram(counts, title=label)
    filename = re.sub(r"[^\w\-]+", "_", label.lower()) + ".png"
    fig.savefig(filename)
    plt.close(fig)

# 엔트로피 계산 및 비교 시각화
labels = list(results.keys())
entropies = [shannon_entropy(results[label]) for label in labels]

plt.figure()
plt.bar(labels, entropies)
plt.xticks(rotation=30)
plt.ylabel("Shannon Entropy")
plt.title("Shannon Entropy Comparison")
plt.tight_layout()
plt.grid(True)
plt.savefig("entropy_comparison.png")
plt.show()

# 유사도 로그 저장
with open("similarity_log.txt", "w", encoding="utf-8") as f:
    f.write("유사도 로그\n------------\n")
    for i in range(len(labels) - 1):
        c1 = results[labels[i]]
        c2 = results[labels[i + 1]]
        all_keys = sorted(set(c1) | set(c2))
        v1 = distribution_vector(c1, all_keys)
        v2 = distribution_vector(c2, all_keys)
        sim = 1 - cosine(v1, v2)
        f.write(f"{labels[i]} vs {labels[i+1]}: {sim:.4f}\n")
